# Streamlit | Streamlit

> [Streamlit](https://docs.streamlit.io/)は、機械学習やデータサイエンス向けの美しいカスタムウェブアプリを簡単に作成し、共有することができるオープンソースのPythonライブラリです。
>
> > [Streamlit](https://docs.streamlit.io/) is an open-source Python library that makes it easy to create and share beautiful,
> > custom web apps for machine learning and data science.

このノートブックでは、`Streamlit`アプリでチャットメッセージの履歴を保存し使用する方法について説明します。`StreamlitChatMessageHistory`は、指定された`key=`を使用して[Streamlitセッションステート](https://docs.streamlit.io/library/api-reference/session-state)にメッセージを保存します。デフォルトのキーは`"langchain_messages"`です。

> This notebook goes over how to store and use chat message history in a `Streamlit` app. `StreamlitChatMessageHistory` will store messages in
> [Streamlit session state](https://docs.streamlit.io/library/api-reference/session-state)
> at the specified `key=`. The default key is `"langchain_messages"`.

* `StreamlitChatMessageHistory`は、Streamlitアプリで実行されている場合にのみ機能します。

  > Note, `StreamlitChatMessageHistory` only works when run in a Streamlit app.

* LangChainに対応する[StreamlitCallbackHandler](/docs/integrations/callbacks/streamlit)にも興味を持つかもしれません。

  > You may also be interested in [StreamlitCallbackHandler](/docs/integrations/callbacks/streamlit) for LangChain.

* Streamlitについてもっと知りたい方は、[入門ドキュメント](https://docs.streamlit.io/library/get-started)をご覧ください。

  > For more on Streamlit check out their
  > [getting started documentation](https://docs.streamlit.io/library/get-started).


完全なアプリケーションの例は[こちらで動作しているものを見ることができます](https://langchain-st-memory.streamlit.app/)。その他の例については、[github.com/langchain-ai/streamlit-agent](https://github.com/langchain-ai/streamlit-agent)でさらに確認できます。

> You can see the [full app example running here](https://langchain-st-memory.streamlit.app/), and more examples in
> [github.com/langchain-ai/streamlit-agent](https://github.com/langchain-ai/streamlit-agent).




In [ ]:
from langchain.memory import StreamlitChatMessageHistory

history = StreamlitChatMessageHistory(key="chat_messages")

history.add_user_message("hi!")
history.add_ai_message("whats up?")

In [ ]:
history.messages

`StreamlitChatMessageHistory`は、通常どおり`ConversationBufferMemory`やチェーン、エージェントに統合することができます。その履歴は、特定のユーザーセッション内でStreamlitアプリが再実行される間、保持されます。ただし、特定の`StreamlitChatMessageHistory`は、ユーザーセッション間で持続されたり共有されたりすることはありません。

> You can integrate `StreamlitChatMessageHistory` into `ConversationBufferMemory` and chains or agents as usual. The history will be persisted across re-runs of the Streamlit app within a given user session. A given `StreamlitChatMessageHistory` will NOT be persisted or shared across user sessions.




In [ ]:
from langchain.memory import ConversationBufferMemory
from langchain.memory.chat_message_histories import StreamlitChatMessageHistory

# Optionally, specify your own session_state key for storing messages
msgs = StreamlitChatMessageHistory(key="special_app_key")

memory = ConversationBufferMemory(memory_key="history", chat_memory=msgs)
if len(msgs.messages) == 0:
    msgs.add_ai_message("How can I help you?")

In [ ]:
from langchain.chains import LLMChain
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate

template = """You are an AI chatbot having a conversation with a human.

{history}
Human: {human_input}
AI: """
prompt = PromptTemplate(input_variables=["history", "human_input"], template=template)

# Add the memory to an LLMChain as usual
llm_chain = LLMChain(llm=OpenAI(), prompt=prompt, memory=memory)

Conversational Streamlitアプリは、再実行のたびに以前のチャットメッセージを再描画することがよくあります。これは`StreamlitChatMessageHistory.messages`を繰り返し処理することで簡単に行うことができます：

> Conversational Streamlit apps will often re-draw each previous chat message on every re-run. This is easy to do by iterating through `StreamlitChatMessageHistory.messages`:




In [ ]:
import streamlit as st

for msg in msgs.messages:
    st.chat_message(msg.type).write(msg.content)

if prompt := st.chat_input():
    st.chat_message("human").write(prompt)

    # As usual, new messages are added to StreamlitChatMessageHistory when the Chain is called.
    response = llm_chain.run(prompt)
    st.chat_message("ai").write(response)

**[最終的なアプリを見る](https://langchain-st-memory.streamlit.app/)。**

> **[View the final app](https://langchain-st-memory.streamlit.app/).**


